# Add the capacity to verify the board

Checking the board visually is fine for debugging but it does not work well if we want to test a lot of board solutions

Goals:
- given a board is it completed ? 
- Are cells left undefined ? 
- Does it holds the clues ?

## Board class from notebook 01

In [81]:
import numpy as np
from enum import Enum

# constants for mark lette
class BoardMark(Enum):
    INIT = -1
    FILLER = 0
    BLACK = 1

class Board:
    '''
    Manage the board.
    '''
    
    def __init__(self, some_clues):
        '''
        board constructor
        '''
        # given parameters
        self.clues = some_clues
        # compute board dimensions
        self.width = len(self.clues['cols']) 
        self.height = len(self.clues['rows']) 
        # create board of type int, initialized with -1
        default_value = BoardMark.INIT.value
        self.states = np.full((self.height, self.width), default_value, dtype=int)
        
    def mark(self, row, col, mark):
        '''
        mark a cell
        '''
        self.states[row, col] = mark.value

    def prettyprint(self):
        '''
        pretty print of the board 
        '''
        print("cols:", end=" ")
        print(*self.clues['cols']) # * unpacks and remove []s
        print("rows:")
        [print(r) for r in self.clues['rows']]
          
        marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
        # for whatever reason lambda does not work here
        def f_marks(v):
            return marks_switcher[v]
        applyall = np.vectorize(f_marks)
        marks = applyall(self.states)
        print(marks)


In [82]:
# clues for mini game
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board = Board(clues)
board.prettyprint()
board.mark(0, 1, BoardMark.FILLER) 
board.mark(1, 1, BoardMark.BLACK) 
board.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]
cols: 2 1
rows:
1
2
[['.' 'x']
 ['.' 'o']]


## norm the clues 

consistency, prepare groupby

In [83]:
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}

In [84]:
assert sum(clues['rows']) == 3

In [85]:
assert sum(clues['rows']) == sum(clues['cols'])

In [86]:
clues = {
    'rows': [1, 3, [1,1], 3, 5],
    'cols': [1, 4, [2,2], 4, 1]
}

In [87]:
#TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [88]:
f_norm_clue = lambda clue: clue if isinstance(clue, list) else [clue]
[f_norm_clue(clue) for clue in [1, 3, [1,1], 3, 5]]

[[1], [3], [1, 1], [3], [5]]

In [89]:
# norm clues
f_norm_clue = lambda clue: clue if isinstance(clue, list) else [clue]
norm_clues = {
    'rows': [f_norm_clue(clue) for clue in clues['rows']],
    'cols': [f_norm_clue(clue) for clue in clues['cols']]
}
norm_clues

{'rows': [[1], [3], [1, 1], [3], [5]], 'cols': [[1], [4], [2, 2], [4], [1]]}

## get number of blacks

In [90]:
from itertools import chain
list(chain.from_iterable(norm_clues['rows']))

[1, 3, 1, 1, 3, 5]

In [91]:
sum(chain.from_iterable(norm_clues['rows']))

14

In [92]:
assert sum(chain.from_iterable(norm_clues['rows'])) == 14

## check numbers of cells left

In [93]:
[ c for c in board.states.reshape(4) if c == -1]

[-1, -1]

In [94]:
count_left = len([ c for c in board.states.reshape(4) if c == -1])
print(f"count_left={count_left}")
assert count_left == 2

count_left=2


## check whether board hold the clues

In [95]:
# clues for mini game
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board = Board(clues)
board.prettyprint()
board.mark(0, 0, BoardMark.BLACK) 
board.mark(0, 1, BoardMark.FILLER) 
board.mark(1, 0, BoardMark.BLACK) 
board.mark(1, 1, BoardMark.BLACK) 
board.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]
cols: 2 1
rows:
1
2
[['o' 'x']
 ['o' 'o']]


In [96]:
lines = board.states
lines

array([[1, 0],
       [1, 1]])

In [97]:
from itertools import groupby

In [98]:
# group cells by value and select groups where group key is 1 (BLACK)
[ [(k, *g) for k,g in groupby(line) if k==1]
  for line in lines
]

[[(1, 1)], [(1, 1, 1)]]

In [99]:
blocks = [ [len(list(g)) for k,g in groupby(line) if k==1]
  for line in lines
]
print(f"blocks={blocks}")
assert blocks == [[1], [2]]

blocks=[[1], [2]]


In [100]:
# we actually need to compare with [1, 2]
[[clue] for clue in clues['rows'] ]

[[1], [2]]

In [101]:
[[clue] for clue in [1, 3, [1,1], 3, 5]]

[[1], [3], [[1, 1]], [3], [5]]

In [102]:
f = lambda clue: clue if isinstance(clue, list) else [clue]
[f(clue) for clue in [1, 3, [1,1], 3, 5]]

[[1], [3], [1, 1], [3], [5]]

In [103]:
f = lambda clue: clue if isinstance(clue, list) else [clue]

In [104]:
[f(clue) for clue in clues['rows'] ]

[[1], [2]]

In [105]:
assert blocks == [f(clue) for clue in clues['rows'] ]

In [106]:
w = 2
h = 2
cols = [ [board.states.reshape(w*h)[c+r*w]
         for r in range(0, h)]
         for c in range(0, w)]
cols

[[1, 1], [0, 1]]

In [107]:
blocks = [ [len(list(g)) for k,g in groupby(line) if k==1]
  for line in cols
]
print(f"blocks={blocks}")
assert blocks == [[2], [1]]

blocks=[[2], [1]]


In [108]:
assert blocks == [f(clue) for clue in clues['cols'] ]

## updated class

In [109]:
%reset -f

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [122]:
import numpy as np
from enum import Enum
from itertools import groupby, chain

# constants for mark lette
class BoardMark(Enum):
    INIT = -1
    FILLER = 0
    BLACK = 1

class Board:
    '''
    Manage the board.
    '''
    
    def __init__(self, some_clues):
        '''
        board constructor
        '''
        # given parameters
        self.clues = some_clues
        
        # normalize clues
        self.norm_clues = self.check_clues(some_clues)
        
        # compute board dimensions
        self.width = len(self.clues['cols']) 
        self.height = len(self.clues['rows']) 
        self.count_cells = self.width * self.height
        self.total_blacks = sum(chain.from_iterable(self.norm_clues['rows']))
        
        # create board of type int, initialized with -1
        default_value = BoardMark.INIT.value
        self.states = np.full((self.height, self.width), default_value, dtype=int)
    
    def check_clues(self, clues):
        f_norm_clue = lambda clue: clue if isinstance(clue, list) else [clue]
        norm_clues = {
            'rows': [f_norm_clue(clue) for clue in clues['rows']],
            'cols': [f_norm_clue(clue) for clue in clues['cols']]
        }
        return norm_clues
        
    def mark(self, row, col, mark):
        '''
        mark a cell
        '''
        self.states[row, col] = mark.value

    def prettyprint(self):
        '''
        pretty print of the board 
        '''
        print("cols:", end=" ")
        print(*self.clues['cols']) # * unpacks and remove []s
        print("rows:")
        [print(r) for r in self.clues['rows']]
          
        marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
        # for whatever reason lambda does not work here
        def f_marks(v):
            return marks_switcher[v]
        applyall = np.vectorize(f_marks)
        marks = applyall(self.states)
        print(marks)

    def count_cells_left(self):
        return len([ c 
                  for c in self.states.reshape(self.count_cells) 
                  if c == BoardMark.INIT.value])


    def does_hold_clues(self):

        rows = self.states
        rows_blocks = [ [len(list(g)) for k,g in groupby(row) if k==1]
          for row in rows
        ]
        rows_status = rows_blocks == self.norm_clues['rows'] 
        
        cols = [ [self.states.reshape(self.count_cells)[c+r*self.width]
         for r in range(0, self.height)]
         for c in range(0, self.width)]
        cols_blocks = [ [len(list(g)) for k,g in groupby(col) if k==1]
          for col in cols
        ]
        cols_status = cols_blocks == self.norm_clues['cols'] 

        return rows_status and cols_status
    
    
    def is_done(self):
        no_cells_left = self.count_cells_left() == 0
        return no_cells_left and self.does_hold_clues()


In [123]:
# clues for mini game
clues1 = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board1 = Board(clues1)
board1.prettyprint()
board1.mark(0, 0, BoardMark.BLACK) 
board1.mark(0, 1, BoardMark.FILLER) 
board1.mark(1, 0, BoardMark.BLACK) 
board1.mark(1, 1, BoardMark.BLACK) 
board1.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]
cols: 2 1
rows:
1
2
[['o' 'x']
 ['o' 'o']]


In [124]:
assert board1.total_blacks == 3

In [125]:
assert board1.count_cells_left() == 0

In [126]:
assert board1.does_hold_clues() 

In [127]:
assert board1.is_done() 

## check 5x5 board

In [128]:
clues1 = {
    'rows': [1, 3, [1,1], 3, 5],
    'cols': [1, 4, [2,2], 4, 1]
}
board1 = Board(clues1)

# row 4 is ooooo - only option for 5
board1.states[4, :] = BoardMark.BLACK.value
# col 2 is ooxoo - only option for [2, 2]
board1.states[:, 2] = BoardMark.BLACK.value
board1.states[2, 2] = BoardMark.FILLER.value
# col 1 and 3 are xoooo - continguous to the placed black
board1.states[:, 1] = BoardMark.BLACK.value
board1.states[0, 1] = BoardMark.FILLER.value
board1.states[:, 3] = BoardMark.BLACK.value
board1.states[0, 3] = BoardMark.FILLER.value
# rows 1 and 3 are xooox - 3 blacks are placed
board1.states[1, 0] = BoardMark.FILLER.value
board1.states[1, 4] = BoardMark.FILLER.value
board1.states[3, 0] = BoardMark.FILLER.value
board1.states[3, 4] = BoardMark.FILLER.value
# row 2 is xoxox - [1, 1] blacks are placed
board1.states[2, 0] = BoardMark.FILLER.value
board1.states[2, 4] = BoardMark.FILLER.value
# row 0 is xoxox - 1 black is placed
board1.states[0, 0] = BoardMark.FILLER.value
board1.states[0, 4] = BoardMark.FILLER.value

board1.prettyprint()


cols: 1 4 [2, 2] 4 1
rows:
1
3
[1, 1]
3
5
[['x' 'x' 'o' 'x' 'x']
 ['x' 'o' 'o' 'o' 'x']
 ['x' 'o' 'x' 'o' 'x']
 ['x' 'o' 'o' 'o' 'x']
 ['o' 'o' 'o' 'o' 'o']]


In [129]:
assert board1.is_done() 

In [130]:
assert board1.total_blacks == 14